In [1]:
#default_exp training

In [24]:
#export
import pandas as pd
import os
from deeplearning_image_classification import data_loading

from sklearn import model_selection, metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [3]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
    
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [4]:
train_csv_path = os.path.join(data_loading.DATA_DIR, 'train_metadata.csv')
test_csv_path = os.path.join(data_loading.DATA_DIR, 'test_metadata.csv')

train_metadata_df = pd.read_csv(train_csv_path)
test_metadata_df = pd.read_csv(test_csv_path)

In [5]:
__, sample_metadata_df = model_selection.train_test_split(train_metadata_df, test_size=1000, random_state=0)

In [6]:
image_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, horizontal_flip=True)
image_size = (224, 224)


sample_image_iterator = image_gen.flow_from_dataframe(sample_metadata_df, batch_size=48, target_size=image_size)
test_image_iterator = image_gen.flow_from_dataframe(test_metadata_df, batch_size=64, target_size=image_size, shuffle=False)


n_classes = len(sample_image_iterator.class_indices)

Found 1000 validated image filenames belonging to 82 classes.
Found 10000 validated image filenames belonging to 82 classes.


# Setting up model

We use pretrained MobileNet model for feature extraction

In [7]:
import keras.applications


base_model = keras.applications.MobileNet(include_top=False, input_shape=(224, 224, 3))
model = keras.Sequential(
    [
        base_model,
        keras.layers.AveragePooling2D((4, 4)),
        keras.layers.Flatten(),
        keras.layers.Dense(n_classes),
        keras.layers.Softmax()
    ])
model.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=keras.optimizers.Adam())

Using TensorFlow backend.


In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 7, 7, 1024)        3228864   
_________________________________________________________________
average_pooling2d_1 (Average (None, 1, 1, 1024)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 82)                84050     
_________________________________________________________________
softmax_1 (Softmax)          (None, 82)                0         
Total params: 3,312,914
Trainable params: 3,291,026
Non-trainable params: 21,888
_________________________________________________________________


In [9]:
model.fit(sample_image_iterator, epochs=10)

Epoch 1/10
21/21 [==============================] - 22s 1s/step - loss: 4.1816
Epoch 2/10
21/21 [==============================] - 10s 458ms/step - loss: 2.4778
Epoch 3/10
21/21 [==============================] - 10s 459ms/step - loss: 1.3892
Epoch 4/10
21/21 [==============================] - 10s 459ms/step - loss: 0.6487
Epoch 5/10
21/21 [==============================] - 10s 458ms/step - loss: 0.3235
Epoch 6/10
21/21 [==============================] - 10s 454ms/step - loss: 0.2225
Epoch 7/10
21/21 [==============================] - 10s 455ms/step - loss: 0.1627
Epoch 8/10
21/21 [==============================] - 10s 458ms/step - loss: 0.1167
Epoch 9/10
21/21 [==============================] - 10s 466ms/step - loss: 0.0916
Epoch 10/10
21/21 [==============================] - 10s 466ms/step - loss: 0.0531


In [10]:
y_test_pred = model.predict(test_image_iterator).argmax(axis=1)

In [21]:
y_test = np.array([test_image_iterator.class_indices[c] for c in test_metadata_df['class']])

## Test set accuracy

In [25]:
metrics.accuracy_score(y_test, y_test_pred)

0.1566